In [2]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import os
import re
import numpy as np
from matplotlib import pyplot as plt
from functools import partial
from collections import OrderedDict
from src import project_dir
from src.nsga2.utils import ErrorMetric, HW_Metric

In [4]:
# Define the path to the directory where logs/results are saved
results_dir = os.path.join(project_dir, 'saved_logs')
# results_dir = os.path.join(project_dir, 'logs')
# Create a list of directories in the 'saved_logs' folder
# This list will only include directories (not files)
resdirs = [os.path.join(results_dir, d)
           for d in os.listdir(results_dir) if os.path.isdir(os.path.join(results_dir, d))]

In [5]:
# Initialize an empty dictionary to hold checkboxes for each directory
checkboxes = {}
# Loop through each directory and create a checkbox widget for it
for directory in resdirs:
    checkboxes[directory] = widgets.Checkbox(
        value=False,  # Set the initial checkbox value to False (unchecked)
        disabled=False,  # The checkbox is not disabled, so it can be interacted with
        description=os.path.basename(directory),  # The label for the checkbox is the directory name
        layout=widgets.Layout(width='500px')  # Set the width of the checkbox widget
    )

# Initialize an empty list to store selected directories
selected_directories = []
# Define a function that will update the list of selected directories based on checkbox values
def select_directories(**kwargs):
    selected_directories.clear()  # Clear the list of previously selected directories
    for key in kwargs:
        if kwargs[key] is True:  # If the checkbox for a directory is checked
            selected_directories.append(key)  # Add the directory to the selected list

# Create an interactive output widget to update selected directories based on user input
out = widgets.interactive_output(select_directories, checkboxes)

# Create a header HTML widget to provide instructions to the user
header = widgets.HTML(
    description="Choose all the directories from which results are gathered:",  # Instruction text
    value="",  # The value is empty since it's just an instruction
    style={'description_width': 'initial'}  # Style to make the description width dynamic
)
# Combine the header and checkboxes into a vertical box layout
dir_checkboxes = widgets.VBox([header, *checkboxes.values()])

# Display the checkboxes and the interactive output
display(dir_checkboxes, out)

Output()

In [6]:
print(selected_directories)

['/home/balaskas/axcarbon/saved_logs/ga_mac14nm___2024.09.05-14.39.42.675', '/home/balaskas/axcarbon/saved_logs/ga_mac7nm___2024.09.05-14.32.34.934']


In [11]:
# Function to read data from a CSV file and return the x and y arrays for plotting
def get_xy_data(csv_file, x_metric, y_metric, remove_first_row=True):
    df = pd.read_csv(csv_file)
    if remove_first_row:
        df = df.drop(0)
    return df[x_metric.name].to_numpy(), df[y_metric.name].to_numpy()

In [13]:
# Function to plot multiple graphs side by side for comparison
def plot_side_by_side(directories, hw_metric, error_metric, use_pareto=False, rm_exact=False,
                      x_log_scale=False, y_log_scale=False):
    # Ensure 'directories' is a list, even if a single directory is provided
    if not isinstance(directories, list):
        directories = [directories]

    # Create a figure with subplots arranged in a single row
    fig, axes = plt.subplots(nrows=1,
                             ncols=len(directories),
                             figsize=(6*len(directories), 4))  # Set the size of the figure
    if not isinstance(axes, np.ndarray):
        axes = [axes]
    # Generate distinct colors for each directory to use in the plots
    colors = plt.cm.rainbow(np.linspace(0, 1, len(directories)))

    for ax, color, directory in zip(axes, colors, directories):
        # Construct the path to the CSV file containing evaluation results
        suffix = '_pareto' if use_pareto else ''
        csv_file = os.path.join(directory, f'eval_results{suffix}.csv')
        # Extract x and y data from the CSV file based on the specified metrics
        x, y = get_xy_data(csv_file, x_metric=error_metric, y_metric=hw_metric, remove_first_row=rm_exact)

        # Set labels for the x and y axes based on the metrics
        ax.set_xlabel(error_metric.name)
        ax.set_ylabel(hw_metric.name)
        if y_log_scale:
            ax.set_yscale('log')
        if x_log_scale:
            ax.set_xscale('log')
        # Plot the data on the current subplot
        ax.plot(x, y,
                marker='x', linestyle='--',
                label=re.search('([\d]+nm)', os.path.basename(directory)).group(1),  # Use part of the directory name as the label
                c=color)  # Set the color of the plot
        ax.grid(visible=True, which='major', axis='both', alpha=0.75)
        ax.grid(visible=True, which='minor', axis='both', alpha=0.25)

    fig.legend()

# Create widgets for selecting the hardware and error metrics
hw_metric_widget = widgets.RadioButtons(options=[metric for metric in HW_Metric])
error_metric_widget = widgets.RadioButtons(options=[metric for metric in ErrorMetric])
pareto_widget = widgets.Checkbox(value=False, disabled=False,
                                 description='Use only Pareto solutions?')
rm_exact_widget = widgets.Checkbox(value=False, disabled=False,
                                   description='Remove the exact solution?')
xlog_widget = widgets.Checkbox(value=False, disabled=False,
                               description='Use log scale in x-axis?')
ylog_widget = widgets.Checkbox(value=False, disabled=False,
                               description='Use log scale in y-axis?')
# Create a partial function of 'plot_side_by_side' with the directories argument fixed
plot_f = partial(plot_side_by_side, directories=selected_directories)
# Create an interactive output widget that updates when the metric selections change
out = widgets.interactive_output(plot_f, {'hw_metric': hw_metric_widget,
                                          'error_metric': error_metric_widget,
                                          'use_pareto': pareto_widget,
                                          'rm_exact': rm_exact_widget,
                                          'x_log_scale': xlog_widget,
                                          'y_log_scale': ylog_widget})
# Arrange the metric selection widgets and the output plot vertically
w = widgets.VBox([
    widgets.HBox([
        error_metric_widget, hw_metric_widget,
        widgets.VBox([
            pareto_widget, rm_exact_widget, xlog_widget, ylog_widget
        ])]),
    out])

# Display the combined widget box
display(w)

In [14]:
# Function to plot multiple datasets on the same axes for comparison
def plot_together(directories, hw_metric, error_metric, use_pareto=False, rm_exact=False,
                  x_log_scale=False, y_log_scale=False):
    # Ensure 'directories' is a list, even if a single directory is provided
    if not isinstance(directories, list):
        directories = [directories]

    fig, ax = plt.subplots(figsize=(6, 4))  # Set the size of the figure
    # Generate distinct colors for each directory to use in the plot
    colors = plt.cm.rainbow(np.linspace(0, 1, len(directories)))

    # Loop through each directory and its corresponding color
    for color, directory in zip(colors, directories):
        # Construct the path to the CSV file containing evaluation results
        suffix = '_pareto' if use_pareto else ''
        csv_file = os.path.join(directory, f'eval_results{suffix}.csv')
        # Extract x and y data from the CSV file based on the specified metrics
        x, y = get_xy_data(csv_file, x_metric=error_metric, y_metric=hw_metric, remove_first_row=rm_exact)

        # Set labels for the x and y axes based on the metrics
        ax.set_xlabel(error_metric.name)
        ax.set_ylabel(hw_metric.name)
        if y_log_scale:
            ax.set_yscale('log')
        if x_log_scale:
            ax.set_xscale('log')
        # Plot the data on the same axis
        ax.plot(x, y,
                marker='x', linestyle='--',
                label=re.search('([\d]+nm)', os.path.basename(directory)).group(1),  # Use part of the directory name as the label
                c=color)  # Set the color of the plot
        ax.grid(visible=True, which='major', axis='both', alpha=0.75)
        ax.grid(visible=True, which='minor', axis='both', alpha=0.25)

    fig.legend()

# Create widgets for selecting the hardware and error metrics
hw_metric_widget = widgets.RadioButtons(options=[metric for metric in HW_Metric])
error_metric_widget = widgets.RadioButtons(options=[metric for metric in ErrorMetric])
pareto_widget = widgets.Checkbox(value=False, disabled=False,
                                 description='Use only Pareto solutions?')
rm_exact_widget = widgets.Checkbox(value=False, disabled=False,
                                   description='Remove the exact solution?')
xlog_widget = widgets.Checkbox(value=False, disabled=False,
                               description='Use log scale in x-axis?')
ylog_widget = widgets.Checkbox(value=False, disabled=False,
                               description='Use log scale in y-axis?')
# Create a partial function of 'plot_together' with the directories argument fixed
plot_f = partial(plot_together, directories=selected_directories)
# Create an interactive output widget that updates when the metric selections change
out = widgets.interactive_output(plot_f, {'hw_metric': hw_metric_widget,
                                          'error_metric': error_metric_widget,
                                          'use_pareto': pareto_widget,
                                          'rm_exact': rm_exact_widget,
                                          'x_log_scale': xlog_widget,
                                          'y_log_scale': ylog_widget})
# Arrange the metric selection widgets and the output plot vertically
w = widgets.VBox([
    widgets.HBox([
        error_metric_widget, hw_metric_widget,
        widgets.VBox([
            pareto_widget, rm_exact_widget, xlog_widget, ylog_widget
        ])]),
    out])

# Display the combined widget box
display(w)